# Setup and Problem

Note: I initially did all of my tests using Ollama models (mistral:7b, gemma3:4b, and qwen3:8b), however, I lost a lot of time trying to setup the structured json outputs for these small models. Due to this I ended up using OpenAI gpt-4o-mini instead to avoid dealing with parsing issues.

For the project I in fact used 4 datasets (PiQA, HellaSwag, BoolQ, and GMS8K)
The reason for this is actually because I originally intended to use a drug-drug interaction task using DrugBank's API, however the data engineering ended up a bit too complicated.
So instead (as said in the project guidelines) I used some open source datasets.

The primary dataset that I wanted to analyse the results over is HellaSwag which is a dataset of common-sense reasoning MCQ, however, I also wanted to have other reasoning datasets in order to cross-compare the different performances. I also felt only doing one of these reasoning tasks would not be sufficient enough given the scope of the mini project. This ended up having some interesting results, so I'm glad that I did (albeit the extra time and costs).

## PiQA :
- Problem: This task evaluates physical commonsense reasoning. The model is given a "goal" and must choose which of two possible solutions is the more practical or correct one.
## HellaSwag:
- Problem: This is a commonsense Natural Language Inference (NLI) task. The model is given a context (the beginning of a sentence) and must choose the most plausible ending from four options
## BoolQ:
- Problem: This is a reading comprehension task requiring a "yes" or "no" answer. The model is given a passage of text and a question about it.
  
and lastly what the model(s) should do well against
## GMS8K:
- Problem: This task evaluates the model's ability to perform multi-step mathematical reasoning to solve grade-school-level word problems.




### Initial test prompt(s)

In [ ]:
// Initial test prompts per task
const INSTRUCTIONS = {
  piqa:
    'You are a classifier for physical commonsense reasoning. ' +
    'Analyze both options carefully and choose the more practical solution. ' +
    'Return ONLY valid JSON: {"label": "A"} or {"label": "B"}.',

  hellaswag:
    'You are a commonsense reasoning classifier. ' +
    'Read the context and determine which ending makes most sense. ' +
    'Return ONLY valid JSON: {"label": "A|B|C|D"}.',

  boolq:
    'You are a reading comprehension system. ' +
    'Read the passage and answer the question truthfully. ' +
    'Return ONLY valid JSON: {"answer": "yes"} or {"answer": "no"}.',

  gsm8k:
    'You are a math problem solver. ' +
    'Solve the problem step by step. ' +
    'Return ONLY valid JSON: {"answer": <number>, "reasoning": "<optional>"}.'
};

### Input Examples

In [3]:
// HellaSwag
import {loadHellaSwag} from "./datasets.ts";

const hellaData = await loadHellaSwag(8);
console.log("HellaSwag loaded:", hellaData.length, "examples");
console.log("Sample:", hellaData[0]);
console.log("Sample:", hellaData[0]);
console.log("Sample:", hellaData[1]);
console.log("Sample:", hellaData[2]);
console.log("Sample:", hellaData[3]);
console.log("Sample:", hellaData[4]);
console.log("Sample:", hellaData[5]);
console.log("Sample:", hellaData[6]);
console.log("Sample:", hellaData[7]);
console.log("Sample:", hellaData[8]);

HellaSwag loaded: 8 examples
Sample: {
  question: "A man is sitting on a roof. he",
  choices: [
    "is using wrap to wrap a pair of skis.",
    "is ripping level tiles off.",
    "is holding a rubik's cube.",
    "starts pulling up roofing on a roof."
  ],
  correct: "D"
}
Sample: {
  question: "A man is sitting on a roof. he",
  choices: [
    "is using wrap to wrap a pair of skis.",
    "is ripping level tiles off.",
    "is holding a rubik's cube.",
    "starts pulling up roofing on a roof."
  ],
  correct: "D"
}
Sample: {
  question: "A lady walks to a barbell. She bends down and grabs the pole. the lady",
  choices: [
    "swings and lands in her arms.",
    "pulls the barbell forward.",
    "pulls a rope attached to the barbell.",
    "stands and lifts the weight over her head."
  ],
  correct: "D"
}
Sample: {
  question: "Two women in a child are shown in a canoe while a man pulls the canoe while standing in the water, with other individuals visible in the background. the chi

In [ ]:
// PiQA
{
    "goal": "How do I ready a guinea pig cage for it's new occupants?",
    "sol1": "Provide the guinea pig with a cage full of a few inches of bedding made of ripped paper strips, you will also need to supply it with a water bottle and a food dish.",
    "sol2": "Provide the guinea pig with a cage full of a few inches of bedding made of ripped jeans material, you will also need to supply it with a water bottle and a food dish.",
    "label": "0"
  }

In [ ]:
// BoolQ
{
    "question": "does ethanol take more energy make that produces",
    "passage": "All biomass goes through at least some of these steps: it needs to be grown, collected, dried, fermented, distilled, and burned. All of these steps require resources and an infrastructure. The total amount of energy input into the process compared to the energy released by burning the resulting ethanol fuel is known as the energy balance (or ``energy returned on energy invested''). Figures compiled in a 2007 report by National Geographic Magazine point to modest results for corn ethanol produced in the US: one unit of fossil-fuel energy is required to create 1.3 energy units from the resulting ethanol. The energy balance for sugarcane ethanol produced in Brazil is more favorable, with one unit of fossil-fuel energy required to create 8 from the ethanol. Energy balance estimates are not easily produced, thus numerous such reports have been generated that are contradictory. For instance, a separate survey reports that production of ethanol from sugarcane, which requires a tropical climate to grow productively, returns from 8 to 9 units of energy for each unit expended, as compared to corn, which only returns about 1.34 units of fuel energy for each unit of energy expended. A 2006 University of California Berkeley study, after analyzing six separate studies, concluded that producing ethanol from corn uses much less petroleum than producing gasoline.",
    "answer": false
  }

In [ ]:
// GMS8K
{
    "question": "Janet\u2019s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
    "answer": 18.0,
    "solution": "Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer\u2019s market.\n#### 18"
  }

### Evaluator(s)

For each of these datasets there is only one evaluator function which does exact match only.

In [4]:
// For 2 or 4 choice multiple choice tasks (PiQA and HellaSwag)
export function makeMCEvaluator(
  config: LLMConfig,
  numChoices: 2 | 4 = 2
): EvalExample<MCExample> {
  return async (instruction: string, ex: MCExample) => {
    const choiceLabels = numChoices === 2 ? ['A', 'B'] : ['A', 'B', 'C', 'D'];
    const optionsText = ex.choices
      .slice(0, numChoices)
      .map((choice, i) => `${choiceLabels[i]}) ${choice}`)
      .join('\n');

    const user =
      `Question: ${ex.question}\n\n` +
      `Options:\n${optionsText}\n\n` +
      `Respond with ONLY the letter of the correct answer (${choiceLabels.join(' or ')}). No explanation.`;

    const { response, tokens } = await chatText(
      config,
      [
        { role: "system", content: instruction },
        { role: "user", content: user }
      ]
    );

    // Extract first uppercase letter from response
    const match = response.match(/[A-D]/);
    const predicted = match ? match[0] : null;

    // If matched yay, else nay
    const score = predicted === ex.correct ? 1.0 : 0.0;

    return { score, tokens };
  };
}

// Evals for PIQA and HellaSwag (2 and 4 choices)
export function makePIQAEvaluator(config: LLMConfig): EvalExample<MCExample> {
  return makeMCEvaluator(config, 2);
}

export function makeHellaSwagEvaluator(config: LLMConfig): EvalExample<MCExample> {
  return makeMCEvaluator(config, 4);
}

// Boolean evaluator for BoolQ
export function makeBoolQEvaluator(config: LLMConfig): EvalExample<BoolQExample> {
  return async (instruction: string, ex: BoolQExample) => {
    const user =
      `Passage: ${ex.passage}\n\n` +
      `Question: ${ex.question}\n\n` +
      `Respond with ONLY "yes" or "no". No explanation.`;

    const { response, tokens } = await chatText(
      config,
      [
        { role: "system", content: instruction },
        { role: "user", content: user }
      ]
    );

    const cleaned = response.trim().toLowerCase();
    const predicted = cleaned.includes('yes') ? 'yes' : 
                     cleaned.includes('no') ? 'no' : null;
        
    // If matched yay, else nay
    const goldAnswer = ex.answer ? "yes" : "no";
    const score = predicted === goldAnswer ? 1.0 : 0.0;

    return { score, tokens };
  };
}

// Numeric answer evaluator for GSM8K
export function makeGSM8KEvaluator(config: LLMConfig): EvalExample<GSM8KExample> {
  return async (instruction: string, ex: GSM8KExample) => {
    const user =
      `${ex.question}\n\n` +
      `Respond with ONLY the final numerical answer. No explanation.`;

    const { response, tokens } = await chatText(
      config,
      [
        { role: "system", content: instruction },
        { role: "user", content: user }
      ]
    );

    // Extract last number from response
    const numbers = response.match(/-?\d+\.?\d*/g);
    const predicted = numbers ? parseFloat(numbers[numbers.length - 1]) : null;
    
    const tolerance = 0.01;
    
    // If matched yay, else nay (w/ FP)
    const score =
      predicted !== null && Math.abs(predicted - ex.answer) < tolerance
        ? 1.0
        : 0.0;

    return { score, tokens };
  };
}

## Optimization Algorithms

As I implemented the algorithms in typescript. Instead of running I am simply calling them from their respective modules along with the evaluator(s).
But for reference here are the defined algorithms from the modules:

# APE

In [5]:
import { z } from "npm:zod";
import { chatText, LLMConfig } from "./callOllama.ts";

// prompt is id + the prompt's text
export type Prompt = { id: string; instruction: string };

// Track best current prompt, its score, and tokens used
export type BestPrompt = { best: Prompt; score: number; tokens: number };

// Evaluator: given an instruction and an example, return scalar score in [0,1] and tokens used.
export type EvalExample<E> = (
  instruction: string,
  ex: E
) => Promise<{ score: number; tokens: number }>;

// Track tokens used for budget
export class TokenMeter {
  total = 0;
  add(n: number) { this.total += Math.max(0, n|0); }
  snapshot() { return this.total; }
}

// UUID for unique prompt IDs to track
function uuid() { return crypto.randomUUID(); }

// Schema of a paraphrased prompt
const ParaphraseSchema = z.object({
  instruction: z.string().min(8).max(8000),
});

/**
 * Paraphrase the given prompt
 * Counts tokens used by the paraphrasing call
 */
export async function paraphraseInstruction(
  config: LLMConfig,
  base: string,
  meter: TokenMeter,
  style = "Be concise. Keep the same schema. Emphasize: return ONLY valid JSON."
): Promise<{ instruction: string; tokens: number }> {
  const system =
    "You rewrite prompts. Preserve intent and constraints; reduce verbosity; DO NOT change the output schema.";
  const user =
    `Rewrite this instruction with the same meaning and constraints. Do not add examples.\n` +
    `STYLE: ${style}\n\n---\n${base}\n---\n` +
    `Return JSON: {"instruction": "<rewritten>"} (no extra fields).`;

  const { response, tokens } = await chatText(
    config,
    [
      { role: "system", content: system },
      { role: "user", content: user },
    ],
  );

  const rewritten = response.trim() || base;

  meter.add(tokens ?? 0);
  return { instruction: rewritten, tokens: tokens ?? 0 };
}

export type APEOptions<E> = {
  config: LLMConfig; // LLM config for paraphrasing
  baseInstruction: string; // starting instruction to paraphrase
  N: number; // number of paraphrases to try
  data: E[]; // data is the eval examples for: PIQA, HellaSwag, BoolQ, GSM8K
  evalExample: EvalExample<E>; // evaluator function for the dataset
};

export async function apeOptimize<E>(opts: APEOptions<E>) {
  const { config, baseInstruction, N, data, evalExample } = opts;
  const meter = new TokenMeter();

  // Create candidates: base + N paraphrases and set uuids + track total tokens (max per prompt is 128)
  const candidates: Prompt[] = [{ id: uuid(), instruction: baseInstruction }];
  for (let i = 0; i < N; i++) {
    const { instruction } = await paraphraseInstruction(config, baseInstruction, meter);
    candidates.push({ id: uuid(), instruction });
  }

  // Evaluate each candidate on the dataset + examples
  let best = candidates[0];
  let bestScore = -1;

  // Loop through candidates and evaluate
  for (const p of candidates) {
    let sum = 0;

    // Loop through eval examples
    for (let i = 0; i < data.length; i++) {
      const { score, tokens } = await evalExample(p.instruction, data[i]);
      meter.add(tokens); // update total tokens
      sum += score;
    }
    const avg = sum / Math.max(1, data.length); // average score over examples
    if (avg > bestScore) {
      best = p; bestScore = avg; // update best prompt if improved
    }
  }

  // Return best prompt, its score, and token usage
  const bestPrompts: BestPrompt[] = [{ best, score: bestScore, tokens: meter.snapshot() }];
  return { best, bestPrompts, meter };
}


# Evolution

In [6]:
import { z } from "npm:zod";
import { chatText, LLMConfig } from "./callOllama.ts";

export type Prompt = { id: string; instruction: string };
export type BestPrompt = { best: Prompt; score: number; tokens: number };
export type EvalExample<E> = (
  instruction: string,
  ex: E
) => Promise<{ score: number; tokens: number }>;

export class TokenMeter {
  total = 0;
  add(n: number) { this.total += Math.max(0, n|0); }
  snapshot() { return this.total; }
  can(budget?: number | null) { return budget == null || this.total < budget; }
}
function uuid() { return crypto.randomUUID(); }
function pick<T>(xs: T[]) { return xs[Math.floor(Math.random() * xs.length)]; }

// ---- Mutators (LLM rewrites of the instruction) ----
const MutSchema = z.object({ instruction: z.string().min(8).max(8000) });

export type Mutator = (
  config: LLMConfig,
  parentInstruction: string,
  meter: TokenMeter
) => Promise<{ instruction: string; tokens: number }>;

// Create a mutator using meta prompting
function makeMutator(title: string, guidance: string): Mutator {
  return async (config, parent, meter) => {
    const system =
      "You rewrite prompts. Preserve intent and output schema. Apply the guidance. No examples; be concise.";
    const user =
      `GUIDANCE: ${guidance}\n` +
      `Rewrite the instruction below. Keep the same JSON schema and constraints.\n---\n${parent}\n---\n` +
      `Return JSON: {"instruction":"<rewritten>"}`;
    const { response, tokens } = await chatText(
      config,
      [{ role: "system", content: system }, { role: "user", content: user }]
    );
    const instruction = response.trim() || parent;
    meter.add(tokens ?? 0);
    return { instruction, tokens: tokens ?? 0 };
  };
}

// A few defaults inspired by Ape + PromptBreeder paper
export const DEFAULT_MUTATORS: Mutator[] = [
  makeMutator("format_strict", "Stress: output ONLY valid JSON; no explanations."),
  makeMutator("tie_break", "If multiple labels seem plausible, choose the MOST SPECIFIC according to the class set."),
  makeMutator("cautious", "Prefer safety and common-sense plausibility when unsure."),
  makeMutator("reason_silent", "Think step-by-step INTERNALLY and NEVER reveal your reasoning."),
];

type FitnessCacheKey = string;

export type EvoOptions<E> = {
  config: LLMConfig;
  seeds: string[]; // Original prompts to evolve from
  data: E[]; // Eval examples for: PIQA, HellaSwag, BoolQ, GSM8K
  evalExample: EvalExample<E>;
  budget: number | null; // hard token budget (counts mutators + eval)
  mutators?: Mutator[]; // default provided
};

export async function evoOptimize<E>(opts: EvoOptions<E>) {
  const { config, seeds, data, evalExample, budget, mutators = DEFAULT_MUTATORS } = opts;
  const meter = new TokenMeter();

  // Initialize set of prompts
  const pop: Prompt[] = (seeds.length ? seeds : ['Return only {"label":"A|B"}'])
    .map(s => ({ id: uuid(), instruction: s }));

  // Cache fitness results
  const fit = new Map<FitnessCacheKey, number>();
  const keyOf = (p: Prompt) => p.instruction;

  async function fitness(p: Prompt): Promise<number> {
    const k = keyOf(p);
    if (fit.has(k)) return fit.get(k)!;

    let sum = 0;
    for (let i = 0; i < data.length; i++) {
      const { score, tokens } = await evalExample(p.instruction, data[i]);
      meter.add(tokens);
      sum += score;

      if (!meter.can(budget)) break; // stop early if budget blown during eval
    }
    const avg = sum / Math.max(1, data.length);
    fit.set(k, avg);
    return avg;
  }

  // From original prompts find the best as starting point
  let best = pop[0];
  let bestScore = await fitness(best);
  const bestPrompts: BestPrompt[] = [{ best, score: bestScore, tokens: meter.snapshot() }];

  // Tournament loop until budget reached
  while (meter.can(budget)) {
    const a = pick(pop), b = pick(pop);
    const fa = await fitness(a);
    if (!meter.can(budget)) break;
    const fb = await fitness(b);
    if (!meter.can(budget)) break;

    const winner = fa >= fb ? a : b;
    const loserIdx = pop.findIndex(x => x.id === (fa >= fb ? b.id : a.id));

    // replace the loser with a mutated child of the winner
    const mut = pick(mutators);
    const { instruction: childInst } = await mut(config, winner.instruction, meter);
    if (!meter.can(budget)) break;

    // Create child prompt with new uuid and replace in population
    const child: Prompt = { id: uuid(), instruction: childInst };
    pop[loserIdx] = child;

    // Evaluate child fitness and update best if improved
    const sChild = await fitness(child);
    if (!meter.can(budget)) break;

    if (sChild > bestScore) { best = child; bestScore = sChild; }
    bestPrompts.push({ best, score: bestScore, tokens: meter.snapshot() });
  }

  return { best, bestPrompts, meter };
}


# Thompson Sampling (NIG PRior/Posterior)

In [7]:
import { z } from "npm:zod";
import { chatText, LLMConfig } from "./callOllama.ts";

export type Prompt = { id: string; instruction: string };
export type BestPrompts = { best: Prompt; score: number; tokens: number };
export type EvalExample<E> = (
  instruction: string,
  ex: E
) => Promise<{ score: number; tokens: number }>;

export class TokenMeter {
  total = 0;
  add(n: number) {
    this.total += Math.max(0, n | 0);
  }
  snapshot() {
    return this.total;
  }
  can(budget?: number | null) {
    return budget == null || this.total < budget;
  }
}
function uuid() {
  return crypto.randomUUID();
}
function randint(n: number) {
  return Math.floor(Math.random() * n);
}

// Create a single mutation of a prompt
const MutSchema = z.object({ instruction: z.string().min(8).max(8000) });
export async function mutateOnce(
  config: LLMConfig,
  parent: string,
  meter: TokenMeter,
  guidance = "Rewrite to be clearer, shorter, and enforce returning ONLY valid JSON."
): Promise<{ instruction: string; tokens: number }> {
  const system =
    "You rewrite prompts. Preserve intent and output schema. No examples; be concise.";
  const user =
    `GUIDANCE: ${guidance}\n` +
    `Rewrite the instruction below. Keep the same JSON schema.\n---\n${parent}\n---\n` +
    `Return JSON: {"instruction":"<rewritten>"}`;
  const { response, tokens } = await chatText(
    config,
    [
      { role: "system", content: system },
      { role: "user", content: user },
    ]
  );
  const instruction = response.trim() || parent;
  meter.add(tokens ?? 0);
  return { instruction, tokens: tokens ?? 0 };
}

// Normal Inverse Gamma = mean, variance prior/posterior
// Prior and posterior are conjugate this means we can simply update hyperparameters as we see data
// If they were'nt conjugate this would mean we would need to approximate the posterior to update it
type Posterior = {
    mu: number; // mean
    kappa: number; // inverse spread of mean
    alpha: number; // variance shape
    beta: number;  // variance scale
};

// Sample from normal and inverse gamma distributions
// mean is derived from variance's stddev
function normal(mean: number, variance: number) {
  const u = Math.random();
  const v = Math.random();
  const z = Math.sqrt(-2 * Math.log(u)) * Math.cos(2 * Math.PI * v);
  return mean + Math.sqrt(Math.max(variance, 1e-12)) * z;
}

// Draw sample variance from inverse gamma
function invGamma(alpha: number, beta: number) {
  // d = shape, c = scale
  const d = alpha - 1 / 3;
  const c = 1 / Math.sqrt(9 * d);
  let v: number;

  // while v is less than 0, re-sample from standard normal
  do {
    const u = Math.random();
    const w = Math.random();
    const z = Math.sqrt(-2 * Math.log(u)) * Math.cos(2 * Math.PI * w);
    // v = (1 + c * z)^3
    v = Math.pow(1 + c * z, 3);
  } while (v <= 0);

  // return beta / (d * v) as sample from InvGamma
  const gammaSample = d * v;
  return beta / gammaSample;
}

export type TSOptions<E> = {
  config: LLMConfig;
  seeds: string[]; // at least one base instruction
  data: E[];
  evalExample: EvalExample<E>;
  budget: number | null; // hard token budget (mutations + eval)
  extraArms?: number; // how many mutated arms to add at start (3 for now)
  prior?: Posterior; // Normal inverse gamma prior hyperparameters
};

export async function tsOptimize<E>(opts: TSOptions<E>) {
  const { config, seeds, data, evalExample, budget, extraArms = 3 } = opts;
  const prior: Posterior = opts.prior ?? {
    mu: 0.5, // prior mean score
    kappa: 1e-3, // prior strength
    alpha: 1.0, // prior shape (flat prior of (1,1) to start)
    beta: 1.0, // prior scale
  };
  const meter = new TokenMeter();

  // Initialize candidate prompts
  const arms: Prompt[] = (
    seeds.length ? seeds : ['Return only {"label":"A|B"}']
  ).map((s) => ({ id: uuid(), instruction: s }));

  // Create extra mutated arms
  for (let i = 0; i < extraArms; i++) {
    const base = arms[0].instruction;
    const { instruction } = await mutateOnce(config, base, meter);
    if (!meter.can(budget)) break;
    arms.push({ id: uuid(), instruction });
  }

  // Add posterior for each arm
  const P = new Map<string, Posterior>();
  arms.forEach((a) => P.set(a.id, { ...prior }));

  // Track best by posterior mean
  let best = arms[0];
  let bestMu = P.get(best.id)!.mu;
  const bestPrompts: BestPrompts[] = [
    { best, score: bestMu, tokens: meter.snapshot() },
  ];

  // Loop until budget exhausted
  while (meter.can(budget)) {
    // Thompson sample each arm
    let chosen: Prompt | null = null;
    let bestTheta = -Infinity;

    for (const a of arms) {
      const p = P.get(a.id)!;
      const sigma2 = invGamma(p.alpha, p.beta);
      const theta = normal(p.mu, sigma2 / p.kappa);
      if (theta > bestTheta) {
        bestTheta = theta;
        chosen = a;
      }
    }

    if (!chosen) break;

    // Evaluate at random a given example
    const idx = randint(data.length);
    const { score, tokens } = await evalExample(chosen.instruction, data[idx]);
    meter.add(tokens);

    // Posterior update after observing score
    // More observations == more certain about mean and variance
    const post = P.get(chosen.id)!;
    const k1 = post.kappa + 1;
    const mu1 = (post.kappa * post.mu + score) / k1;
    const a1 = post.alpha + 0.5;
    const b1 = post.beta + (post.kappa * (score - post.mu) ** 2) / (2 * k1);
    P.set(chosen.id, { mu: mu1, kappa: k1, alpha: a1, beta: b1 });

    // Track best by posterior mean
    // Update best if improved
    best = arms.reduce(
      (acc, cur) => (P.get(cur.id)!.mu > P.get(acc.id)!.mu ? cur : acc),
      best
    );
    bestMu = P.get(best.id)!.mu;
    bestPrompts.push({ best, score: bestMu, tokens: meter.snapshot() });

    if (!meter.can(budget)) break;
  }

  return { best, bestPrompts, meter, posterior: P };
}


## Running the code
For running the code I simply passed the following

In [ ]:
%deno run --allow-all main.ts --provider openai --model gpt-4o-mini --examples 8

## Visualisations + Reflection in the Python notebook
I put the rest in the python notebook as it was easier to just plot the results in matplotlib